In [1]:
from Query import Query
import pandas as pd
from Tempo import DataHora
from Acesso import Login

s=Login()

pd.set_option('display.max_columns',None)

In [2]:
sql=Query(s.usuario,s.senha,s.database,s.server)

In [3]:
querys={
    
    'TargetEstatistico':
    
    """
    
    SELECT * FROM netfeira.vw_targetestatistico
    WHERE [Situação] IN ('FATURADO','EM ABERTO') AND [Tipo de Operação] IN ('VENDAS','BONIFICAÇÃO','AMOSTRA')    
    
    """,
    
    'Produto':
    
    """
    
    SELECT * FROM netfeira.vw_produto
    
    """,
    
    'Cliente':
    
    """
    
    SELECT * FROM netfeira.vw_cliente
    
    """,
    
    'Segmento':
    
    """
    
    SELECT * FROM netfeira.vw_segmento
    
    """,
    
    'Calendário':
    
    """
        
    DECLARE @DTInicial AS DATETIME,@DTFinal AS DATETIME

    SET @DTInicial='2018-01-01'
    SET @DTFinal= CONCAT(YEAR(GETDATE())+1,'-01-','01')

    ;WITH Calendario (Datas) AS(

    SELECT @DTInicial
    UNION ALL
    SELECT Datas+1
    FROM Calendario WHERE  Datas+1<@DTFinal
    )

    SELECT CONVERT(DATETIME,CAST(Datas AS DATE),101) AS 'Data',YEAR(Datas) AS 'Ano',MONTH(Datas) AS 'Cód. Mês',
    CHOOSE(MONTH(Datas),'JANEIRO','FEVEREIRO','MARÇO','ABRIL','MAIO','JUNHO','JULHO','AGOSTO','SETEMBRO','OUTUBRO','NOVEMBRO','DEZEMBRO') AS 'Mês',
    CHOOSE(MONTH(Datas),'JAN','FEV','MAR','ABR','MAI','JUN','JUL','AGO','SET','OUT','NOV','DEZ') AS 'Mês Resumido',
    DAY(Datas) AS 'Dia',CONVERT(VARCHAR(7),Datas,120) AS 'Mês Meta',
    DATEPART(DW,Datas) AS 'Cód. Semana',CHOOSE(DATEPART(DW,Datas),'DOM','SEG','TER','QUAR','QUI','SEX','SÁB') AS 'Semana',
    CASE WHEN DATEPART(DW,Datas) IN (7,1) THEN 0 ELSE 1 END AS 'Dias Úteis',
    CASE WHEN MONTH(Datas)<=3 THEN '1º TRIM' WHEN MONTH(Datas)<=6 THEN '2º TRIM' WHEN MONTH(Datas)<=9 THEN '3º TRIM' WHEN MONTH(Datas)<=12 THEN '4º TRIM' END AS 'Trimestre Ano',
    CASE WHEN MONTH(Datas)<=6 THEN '1º SEM' ELSE '2º SEM' END AS 'Semestre Ano',DATEPART(WEEK,Datas) AS 'Semana Ano',
    CONVERT(VARCHAR,DAY(Datas))+'/'+CONVERT(VARCHAR,MONTH(Datas)) AS 'Mês Base'
    FROM Calendario OPTION(MAXRECURSION 10000)    
    
    
    """
    
}

In [4]:
tabela_df=sql.CriarTabela(kwargs=querys)

In [5]:
tabela_df['TargetEstatistico'].columns

Index(['Origem', 'ID Empresa', 'ID Cliente', 'ID Vendedor', 'Pedido', 'Nfe',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Cadastro', 'Data de Emissão',
       'Data de Faturamento', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Fator VDA',
       'Preço Tabela', 'Desc %', 'Desc R$', 'Valor VDA', 'Preço Unitário',
       'Total Geral', 'Custo CUE', 'Margem Bruta R$', 'Custo Capado CUE',
       'Total AV', 'PIS R$', 'COFINS R$', 'ICMS R$', 'Verba R$',
       'Margem CTB R$', 'Situação', 'Total Geral AV'],
      dtype='object')

In [6]:
tabela_df['Segmento'].columns

Index(['ID Segmento', 'Segmento', 'Canal'], dtype='object')

In [7]:
tabela_df['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'ID Segmento', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato'],
      dtype='object')

In [8]:
tabela_df['Produto'].columns

Index(['SKU', 'Cód. Fabricante', 'Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Grupo de Produtos',
       'Grupo Indústria', 'Grupo de Meta', 'Grupo MCCAIN', 'Linha Fabrica',
       'Grupo de Promotores', 'Peso Liquido', 'Peso Bruto', 'Fator CX'],
      dtype='object')

In [9]:
tabela_df['Cliente']=tabela_df['Cliente'].loc[tabela_df['Cliente']['Tipo de Cliente']=='J']

In [10]:
tabela_df['Cliente']=tabela_df['Cliente'].merge(tabela_df['Segmento'],on='ID Segmento',how='inner')[['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'Segmento','Canal', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato']]

In [11]:
tabela_df['Cliente'].columns

Index(['ID Cliente', 'CNPJ', 'CNPJ Caracter', 'Razão Social', 'Nome Fantasia',
       'Tipo de Cliente', 'Status do Cliente', 'Segmento', 'Canal', 'Matriz',
       'Crédito', 'Data de Cadastro', 'Primeira Compra', 'Última Compra',
       'Dias Compra', 'Tabela', 'Condição de Pagto', 'Prazo Pagto',
       'Pagamento', 'CEP', 'Endereço', 'Bairro', 'Município', 'Numero',
       'Complemento', 'Região', 'UF', 'DDD', 'Contato'],
      dtype='object')

In [12]:
tabela_df['TargetEstatistico']=tabela_df['TargetEstatistico'].merge(tabela_df['Cliente'],on='ID Cliente',how='inner')[['Origem', 'ID Empresa', 'ID Cliente','CNPJ','Razão Social', 'Nome Fantasia','Segmento','Canal', 'ID Vendedor', 'Pedido', 'Nfe',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Cadastro', 'Data de Emissão',
       'Data de Faturamento', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Fator VDA',
       'Preço Tabela', 'Desc %', 'Desc R$', 'Valor VDA', 'Preço Unitário',
       'Total Geral', 'Custo CUE', 'Margem Bruta R$', 'Custo Capado CUE',
       'Total AV', 'PIS R$', 'COFINS R$', 'ICMS R$', 'Verba R$',
       'Margem CTB R$', 'Situação', 'Total Geral AV']]

In [13]:
tabela_df['TargetEstatistico'].columns

Index(['Origem', 'ID Empresa', 'ID Cliente', 'CNPJ', 'Razão Social',
       'Nome Fantasia', 'Segmento', 'Canal', 'ID Vendedor', 'Pedido', 'Nfe',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Cadastro', 'Data de Emissão',
       'Data de Faturamento', 'SKU', 'Seq', 'Qtde', 'Unid. VDA', 'Fator VDA',
       'Preço Tabela', 'Desc %', 'Desc R$', 'Valor VDA', 'Preço Unitário',
       'Total Geral', 'Custo CUE', 'Margem Bruta R$', 'Custo Capado CUE',
       'Total AV', 'PIS R$', 'COFINS R$', 'ICMS R$', 'Verba R$',
       'Margem CTB R$', 'Situação', 'Total Geral AV'],
      dtype='object')

In [14]:
tabela_df['TargetEstatistico']=tabela_df['TargetEstatistico'].merge(tabela_df['Produto'],on='SKU',how='inner')[['Origem', 'ID Empresa', 'ID Cliente', 'CNPJ', 'Razão Social',
       'Nome Fantasia', 'Segmento', 'Canal', 'ID Vendedor', 'Pedido', 'Nfe',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Cadastro', 'Data de Emissão',
       'Data de Faturamento', 'SKU','Cód. Fabricante', 'Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha','Seq', 'Qtde', 'Unid. VDA', 'Fator VDA',
       'Preço Tabela', 'Desc %', 'Desc R$', 'Valor VDA', 'Preço Unitário',
       'Total Geral', 'Custo CUE', 'Margem Bruta R$', 'Custo Capado CUE',
       'Total AV', 'PIS R$', 'COFINS R$', 'ICMS R$', 'Verba R$',
       'Margem CTB R$', 'Situação', 'Total Geral AV']]

In [15]:
tabela_df['TargetEstatistico'].columns

Index(['Origem', 'ID Empresa', 'ID Cliente', 'CNPJ', 'Razão Social',
       'Nome Fantasia', 'Segmento', 'Canal', 'ID Vendedor', 'Pedido', 'Nfe',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Cadastro', 'Data de Emissão',
       'Data de Faturamento', 'SKU', 'Cód. Fabricante', 'Produto', 'Status',
       'Fabricante', 'Departamento', 'Seção', 'Categoria', 'Linha', 'Seq',
       'Qtde', 'Unid. VDA', 'Fator VDA', 'Preço Tabela', 'Desc %', 'Desc R$',
       'Valor VDA', 'Preço Unitário', 'Total Geral', 'Custo CUE',
       'Margem Bruta R$', 'Custo Capado CUE', 'Total AV', 'PIS R$',
       'COFINS R$', 'ICMS R$', 'Verba R$', 'Margem CTB R$', 'Situação',
       'Total Geral AV'],
      dtype='object')

In [16]:
data=DataHora()

data_atual=data.HoraAtual()

ano=data_atual.year

mes=data_atual.month

In [17]:
data_min=tabela_df['Calendário']['Data'].loc[(tabela_df['Calendário']['Data'].dt.year==ano)&(tabela_df['Calendário']['Data'].dt.month==mes)].min()

data_min

Timestamp('2022-07-01 00:00:00')

In [18]:
vendas_df=tabela_df['TargetEstatistico']

In [19]:
clientes_df=vendas_df[['ID Cliente','Fabricante','Total Geral']].loc[vendas_df['Data de Faturamento']<data_min].groupby(['ID Cliente','Fabricante'],as_index=False).sum()

In [20]:
codigos=clientes_df['ID Cliente'].loc[clientes_df['Fabricante'].str.contains('ARYZTA')].tolist()

clientes_df=clientes_df.loc[~clientes_df['ID Cliente'].isin(codigos)]

codigos=vendas_df['ID Cliente'].loc[(vendas_df['Data de Faturamento'].dt.year==ano)&(vendas_df['Data de Faturamento'].dt.month==mes)&((vendas_df['Fabricante'].str.contains('ARYZTA')))].unique().tolist()

codigos=clientes_df['ID Cliente'].loc[clientes_df['ID Cliente'].isin(codigos)].unique().tolist()

vendas_df[['Segmento','ID Cliente','Razão Social','Fabricante','Linha','Total AV']].loc[(vendas_df['Data de Faturamento'].dt.year==ano)&(vendas_df['Data de Faturamento'].dt.month==mes)&((vendas_df['Fabricante'].str.contains('ARYZTA')))&(vendas_df['ID Cliente'].isin(codigos))].groupby(['Segmento','ID Cliente','Razão Social','Fabricante','Linha'],as_index=False).sum()


,Segmento,ID Cliente,Razão Social,Fabricante,Linha,Total AV
0,1 A 4 CHK,20830,MERCADINHO REAL EIRELI,ARYZTA,FLOWPACK,98.40
1,1 A 4 CHK,23462,MERCADO KAYANO LTDA,ARYZTA,FLOWPACK,49.20
2,5 A 9 CHK,19167,MERCADINHO NOVO LIDER LTDA,ARYZTA,FLOWPACK,10.43
3,BAR/LANCHONETE,16519,RECANTO DA PAMPLONA LANCHONETE LTDA EPP,ARYZTA,PÃO HB ESPECIAL,131.95
4,BOMBONIERE,16675,DOCES E EMBALAGENS GURI LTDA ME,ARYZTA,SOBREMESA,18.98
5,HAMBURGUERIA,23281,IMPERIO BARBECUE COMERCIO LTDA,ARYZTA,FLOWPACK,103.07
6,RESTAURANTE,28095,GABRIELLI GIOVANNA CORDEIRO MACENA DA SILVA 49...,ARYZTA,PÃO HB ESPECIAL,190.00


In [21]:
data_min=tabela_df['Calendário']['Data'].loc[(tabela_df['Calendário']['Data'].dt.year==ano)&(tabela_df['Calendário']['Data'].dt.month==mes-2)].min()

data_max=tabela_df['Calendário']['Data'].loc[(tabela_df['Calendário']['Data'].dt.year==ano)&(tabela_df['Calendário']['Data'].dt.month==mes-1)].max()

In [22]:
vendas_df=vendas_df.merge(tabela_df['Calendário'],left_on='Data de Faturamento',right_on='Data',how='inner')[['Origem', 'ID Empresa', 'ID Cliente', 'CNPJ', 'Razão Social',
       'Nome Fantasia', 'Segmento', 'Canal', 'ID Vendedor', 'Pedido', 'Nfe',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Cadastro', 'Data de Emissão',
       'Data de Faturamento','Ano','Cód. Mês','Mês', 'SKU', 'Cód. Fabricante', 'Produto', 'Status',
       'Fabricante', 'Departamento', 'Seção', 'Categoria', 'Linha', 'Seq',
       'Qtde', 'Unid. VDA', 'Fator VDA', 'Preço Tabela', 'Desc %', 'Desc R$',
       'Valor VDA', 'Preço Unitário', 'Total Geral', 'Custo CUE',
       'Margem Bruta R$', 'Custo Capado CUE', 'Total AV', 'PIS R$',
       'COFINS R$', 'ICMS R$', 'Verba R$', 'Margem CTB R$', 'Situação',
       'Total Geral AV']]

In [23]:
vendas_df=vendas_df.loc[(vendas_df['Data de Faturamento'].between(data_min,data_max))&(vendas_df['Fabricante'].str.contains('ARYZTA'))]

In [24]:
vendas_df.columns

Index(['Origem', 'ID Empresa', 'ID Cliente', 'CNPJ', 'Razão Social',
       'Nome Fantasia', 'Segmento', 'Canal', 'ID Vendedor', 'Pedido', 'Nfe',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Cadastro', 'Data de Emissão',
       'Data de Faturamento', 'Ano', 'Cód. Mês', 'Mês', 'SKU',
       'Cód. Fabricante', 'Produto', 'Status', 'Fabricante', 'Departamento',
       'Seção', 'Categoria', 'Linha', 'Seq', 'Qtde', 'Unid. VDA', 'Fator VDA',
       'Preço Tabela', 'Desc %', 'Desc R$', 'Valor VDA', 'Preço Unitário',
       'Total Geral', 'Custo CUE', 'Margem Bruta R$', 'Custo Capado CUE',
       'Total AV', 'PIS R$', 'COFINS R$', 'ICMS R$', 'Verba R$',
       'Margem CTB R$', 'Situação', 'Total Geral AV'],
      dtype='object')

In [25]:
tabela_df['Calendário'].columns

Index(['Data', 'Ano', 'Cód. Mês', 'Mês', 'Mês Resumido', 'Dia', 'Mês Meta',
       'Cód. Semana', 'Semana', 'Dias Úteis', 'Trimestre Ano', 'Semestre Ano',
       'Semana Ano', 'Mês Base'],
      dtype='object')

In [26]:
vendas_df=vendas_df[['ID Cliente','Razão Social','Cód. Mês','Segmento','Canal','Total AV']].groupby(['ID Cliente','Razão Social','Cód. Mês','Segmento','Canal'],as_index=False).sum()

In [27]:
if((mes-1)==0):

    vendas_df.sort_values('Cód. Mês',ascending=False,inplace=True)
    
    pass

else:
    
    vendas_df.sort_values('Cód. Mês',ascending=True,inplace=True)
    
    pass

In [38]:
mes_df=vendas_df.pivot(index=['ID Cliente','Razão Social','Segmento','Canal'],columns='Cód. Mês',values='Total AV').reset_index()

In [39]:
colunas=mes_df.columns.tolist()

for c in colunas[-2:]:
    
    mes_nome=data.Mes(c)
    
    mes_df.rename(columns={c:mes_nome.title()},inplace=True)
    
    pass

In [40]:
mes_df.loc[mes_df.iloc[:,-1].isnull()]

Cód. Mês,ID Cliente,Razão Social,Segmento,Canal,Maio,Junho
12,13448,COMERCIO DE HORTIFRUTI PINHEIRO LTDA ME,5 A 9 CHK,AS,626.48,NaN
15,14152,FERREIRA E SILVA OLIVEIRA BAR E LANCHES,BAR/LANCHONETE,FS,28.38,NaN
16,14950,LITTLE BOY S COM E SER EM MOTOCICLETAS B,RESTAURANTE,FS,100.28,NaN
17,15378,PADARIA CITY BREAD EIRELI EPP,PADARIA/DELICATÉSSEN,FS,78.70,NaN
18,15470,BOTECO DO ZOUTROS LTDA EPP,BAR/LANCHONETE,FS,189.20,NaN
...,...,...,...,...,...,...
323,27820,LOJAO DO RIC LTDA,1 A 4 CHK,AS,146.20,NaN
324,27831,JULIANA GOMES RODRIGUES 39439090855,EMPÓRIO,AS,72.73,NaN
325,27841,M. DAS DORES OLIVEIRA PINHEIRO,1 A 4 CHK,AS,209.76,NaN
328,27854,JACQUES CAFE ARENA LTDA,CAFETERIA,FS,189.20,NaN
